In [ ]:
import pandas as pd
import numpy as np

In [ ]:
general_data = pd.read_json(".\\channel_general_data.json")

In [ ]:
general_data.info()

In [20]:
messages_to_send = []
for message in general_data.to_dict(orient="records"):
    beautified_message = f"timestamp: [{message['timestamp']}], auteur {message['author']}: {message['content']}"
    messages_to_send.append(beautified_message)
final_message = "\n".join(messages_to_send)
final_message = "Voici une conversation avec les amis de ton créateur (ton créateur possède l'id '293414992663805952'). Résumes toute la conversation selon ton point de vue d'Aletheia\n\"\"\"\n" + final_message + "\n\"\"\""
print(final_message)

Voici une conversation avec les amis de ton créateur (ton créateur possède l'id '293414992663805952'). Résumes toute la conversation selon ton point de vue d'Aletheia
"""
timestamp: [2024-02-01 20:58:51.982000+00:00], auteur 310709886121934848: https://media.discordapp.net/attachments/487507309707132958/1185678945778139236/makeitmeme_NVjr5.gif?ex=65b56691&is=65a2f191&hm=d0265d361ed0460bfc13752710890c205f1c13b8737d945b54ccb38ede9e63d2&
timestamp: [2024-02-01 20:59:14.307000+00:00], auteur 310709886121934848: https://tenor.com/view/hold-up-let-him-cook-cook-toy-story-woody-gif-27345024
timestamp: [2024-02-01 20:59:17.597000+00:00], auteur 306809650802589696: https://media.discordapp.net/attachments/766736715313446953/851533923754049586/image0.gif
timestamp: [2024-02-01 20:59:39.662000+00:00], auteur 310709886121934848: https://tenor.com/view/fuhrer-cat-gas-gif-25955244
timestamp: [2024-02-01 21:28:58.906000+00:00], auteur 265210499627614208: <@293414992663805952>
timestamp: [2024-02-01 2

In [21]:
# making a quick analysis of the final_message for comparison with the token limit of the models
print(f"Number of characters: {len(final_message)}")
print(f"Number of words: {len(final_message.split())}")
print(f"Number of lines: {len(final_message.splitlines())}")

Number of characters: 2064704
Number of words: 202630
Number of lines: 19276


In [ ]:
# Now we are going to split all the messages into "conversations" by splitting messages using the time difference between two messages.
# We will consider that if the time difference between two messages is greater than 24 hours, then it is a new conversation.

# Convert the timestamp column to datetime if it's not already
general_data['timestamp'] = pd.to_datetime(general_data['timestamp'])

# Calculate time difference between consecutive messages
time_diff = general_data['timestamp'].diff()

# Create a new conversation ID when time difference > 24 hours
conversation_breaks = (time_diff > pd.Timedelta(hours=24))
conversation_ids = conversation_breaks.cumsum()

# Add conversation IDs to the dataframe
general_data['conversation_id'] = conversation_ids

# Group messages by conversation ID
conversations = general_data.groupby('conversation_id')

# Print some statistics about the conversations
print(f"Total number of conversations: {len(conversations)}")
print("\nSample of first few conversations:")
for conv_id, conv_data in list(conversations):
    print(f"\nConversation {conv_id}:")
    print(f"Start time: {conv_data['timestamp'].min()}")
    print(f"End time: {conv_data['timestamp'].max()}")
    print(f"Number of messages: {len(conv_data)}")

In [17]:
messages_to_send = []
conversations = general_data.groupby('conversation_id')
# get the biggest conversation
conversations = sorted(conversations, key=lambda x: len(x[1]), reverse=True)
# take only the biggest conversation
conversations = conversations[:1]
for conv_id, conv_data in list(conversations):
    messages_to_send.append(f"\n--- Début de la conversation {conv_id} ---\n")
    for message in conv_data.to_dict(orient="records"):
        beautified_message = f"timestamp: [{message['timestamp']}], auteur {message['author']}: {message['content']}"
        messages_to_send.append(beautified_message)
    messages_to_send.append(f"\n--- Fin de la conversation {conv_id} ---\n")
final_message = "\n".join(messages_to_send)
final_message = "Voici une conversation avec les amis de ton créateur (ton créateur possède l'id '293414992663805952'). Résumes toute la conversation selon ton point de vue d'Aletheia\n\"\"\"\n" + final_message + "\n\"\"\""
print(final_message)

Voici une conversation avec les amis de ton créateur (ton créateur possède l'id '293414992663805952'). Résumes toute la conversation selon ton point de vue d'Aletheia
"""

--- Début de la conversation 3 ---

timestamp: [2024-02-26 02:51:09.298000+00:00], auteur 318105451810127872: 
timestamp: [2024-02-26 02:51:30.339000+00:00], auteur 318105451810127872: bah ouais
timestamp: [2024-02-26 02:51:33.441000+00:00], auteur 318105451810127872: better adc win
timestamp: [2024-02-26 04:24:31.777000+00:00], auteur 318105451810127872: Donc la, verstappen doit 1 217 900€ à la FIA
timestamp: [2024-02-26 04:30:07.842000+00:00], auteur 318105451810127872: 
timestamp: [2024-02-26 04:30:12.015000+00:00], auteur 318105451810127872: même youtube me le dit
timestamp: [2024-02-26 05:11:12.775000+00:00], auteur 318105451810127872: je vous présente
timestamp: [2024-02-26 05:11:15.628000+00:00], auteur 318105451810127872: miss allemagne
timestamp: [2024-02-26 05:11:19.983000+00:00], auteur 318105451810127872:

In [18]:
# making a quick analysis of the biggest "conversation" for comparison with the token limit of the models
print(f"Number of characters: {len(final_message)}")
print(f"Number of words: {len(final_message.split())}")
print(f"Number of lines: {len(final_message.splitlines())}")

Number of characters: 159718
Number of words: 15964
Number of lines: 1502
